In [1]:
import torch
from torch_geometric.data import Data
from torch.utils.data import Dataset
import json
from gensim.models.keyedvectors import KeyedVectors
import numpy as np
from utils import flattenStructure

In [2]:
from data import *
dataset = semeval2017Dataset(dataPath='../dataset/semeval2017-task8/', 
                             type='train',
                             w2vPath='../dataset/glove/',
                             w2vDim=25)

In [4]:
from torch.utils.data import DataLoader
loader = DataLoader(dataset, shuffle=True, num_workers=4)

In [5]:
data = dataset[0]

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, inputDim, hiddenDim, outDim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(inputDim, hiddenDim)
        self.conv2 = GCNConv(hiddenDim + inputDim, outDim)

    def forward(self, data):
        posts, edge_index, rootIndex = data.x, data.edgeIndex, data.rootIndex # posts(n, inputDim)
        
        conv1Out = self.conv1(posts, edge_index)
        postRoot = torch.clone(posts[rootIndex])
        postRoot = postRoot.repeat(posts.shape[0], 1)
        conv1Root = conv1Out[rootIndex]

        conv2In = torch.cat([conv1Out, postRoot], dim=1)
        conv2In = F.relu(conv2In)
        conv2In = F.dropout(conv2In, training=self.training)
        conv2Out = self.conv2(conv2In, edge_index)
        conv2Out = F.relu(conv2Out)

        conv1Root = conv1Root.repeat(posts.shape[0], 1)
        feature = torch.cat([conv1Root, conv2Out], dim=1)
        feature = feature.view(1, feature.shape[0], feature.shape[1])
        # avg_pool1d要求的输入是(batch, in_channels, *)，设置stride,padding为1，使得卷积不会改变特征维度
        feature = F.avg_pool1d(feature, kernel_size=3, stride=1, padding=1)
        return feature.view(feature.shape[1], feature.shape[2])
    
    # 更换计算设备
    def set_device(self, device: torch.device) -> torch.nn.Module:
        _model = self.to(device)
        _model.device = device
        return _model
    # 保存模型
    def save(self, path: str):
        torch.save(self.state_dict(), path)
    # 加载模型
    def load(self, path: str):
        self.load_state_dict(torch.load(path))

class BiGCN(torch.nn.Module):
    def __init__(self, inputDim, hiddenDim, convOutDim, NumRumorTag):
        super(BiGCN, self).__init__()
        self.TDGCN = GCN(inputDim, hiddenDim, convOutDim)
        self.BUGCN = GCN(inputDim, hiddenDim, convOutDim)
        self.fc=torch.nn.Linear((convOutDim + hiddenDim) * 2, NumRumorTag)

    def forward(self, dataTD, dataBU):
        TDOut = self.TDGCN(dataTD)
        BUOut = self.BUGCN(dataBU)
        feature = torch.cat((TDOut, BUOut), dim=1)
        p = self.fc(feature)
        return p[dataTD.rootIndex]

    # 更换计算设备
    def set_device(self, device: torch.device) -> torch.nn.Module:
        _model = self.to(device)
        _model.device = device
        return _model
    # 保存模型
    def save(self, path: str):
        torch.save(self.state_dict(), path)
    # 加载模型
    def load(self, path: str):
        self.load_state_dict(torch.load(path))


class ABGCN(nn.Module):
    def __init__(self,
                 w2vDim: int,
                 s2vDim: int,
                 gcnHiddenDim: int,
                 rumorFeatureDim: int,
                 numRumorTag: int,
                 batchSize = 1,
                 s2vMethon = 'a',
                 numLstmLayer = 2):
        super().__init__()

        self.w2vDim = w2vDim
        self.s2vDim = s2vDim
        self.gcnHiddenDim = gcnHiddenDim
        self.rumorFeatureDim = rumorFeatureDim
        self.s2vMethon = s2vMethon
        self.batchSize = batchSize
        self.numRumorTag = numRumorTag
        self.device = 'cpu'

        # ==使用biLSTM获取post的向量表示==
        if self.s2vMethon == 'l':
            self.numLstmLayer = numLstmLayer
            self.lstm = nn.LSTM(input_size = self.w2vDim,
                                hidden_size = self.s2vDim,
                                num_layers = self.numLstmLayer,
                                bidirectional=True)
            self.h0 = nn.Parameter(torch.randn((2 * 2, self.batchSize, self.s2vDim)))
            self.c0 = nn.Parameter(torch.randn((2 * 2, self.batchSize, self.s2vDim)))
            self.s2vDim *= 2 # 由于使用BiDirect所以s2vDim的维度会扩大1倍
        # ==使用Attention==
        else:
            pass
        
        self.biGCN = BiGCN(self.s2vDim, self.gcnHiddenDim, self.rumorFeatureDim, self.numRumorTag)

    def forward(self, data):
        # 各节点的特征表示，此时是word2vec的形式
        nodeFeature = data['nodeFeature']

        # 把w2v转化成s2v作为节点的特征
        s2v = []
        if self.s2vMethon == 'l':
            for w2v in nodeFeature:
                w2v = w2v.view((len(w2v), 1, -1)).to(self.device)
                sentenceHidden, _ = self.lstm(w2v, (self.h0, self.c0))
                # 仅取出最后一层的隐状态作为s2v
                s2v.append(sentenceHidden[-1].view(1, len(w2v[-1]), -1))
            s2v = torch.cat(s2v, dim=0)
        else:
            pass

        # GCN处理
        s2v = s2v.view(s2v.shape[0], -1)
        dataTD = Data(x = s2v.to(self.device), 
                      edgeIndex = data['edgeIndexTD'].to(self.device), 
                      rootIndex = data['threadIndex'])
        dataBU = Data(x = s2v.to(self.device), 
                      edgeIndex = data['edgeIndexBU'].to(self.device), 
                      rootIndex = data['threadIndex'])
        feature = self.biGCN(dataTD, dataBU) # feature.shape = (4,)
        
        return feature
        

    # 更换计算设备
    def set_device(self, device: torch.device) -> torch.nn.Module:
        _model = self.to(device)
        _model.device = device
        return _model
    # 保存模型
    def save(self, path: str):
        torch.save(self.state_dict(), path)
    # 加载模型
    def load(self, path: str):
        self.load_state_dict(torch.load(path))

model = ABGCN(w2vDim = 25,
              s2vDim = 32,
              gcnHiddenDim = 32,
              rumorFeatureDim = 32,
              numRumorTag = 4,
              s2vMethon = 'l')
model = model.set_device(torch.device('cuda'))
        

In [25]:
feature = model.forward(data)

In [26]:
print(feature)

tensor([-0.0277,  0.0215, -0.0850,  0.0056], device='cuda:0',
       grad_fn=<SelectBackward0>)
